In [ ]:
import scanpy as sc
import decoupler as dc 

In [ ]:
adata = sc.read_h5ad("adata_solo_annotated_all.h5ad")

In [ ]:
adata

In [ ]:
sc.set_figure_params(figsize=(6, 6))

In [ ]:
sc.pl.umap(adata, color=["cell_type","origin"], frameon=False, save="umap_all", legend_loc="on data",legend_fontsize=7 )

In [ ]:
sc.pl.umap(adata, color=["Ifng","Cxcr3"], frameon=False,gene_symbols="gene_name", save= "ifng_cxcr3",legend_loc="on data",legend_fontsize=7 , vmax="p99", cmap="Reds",add_outline=True)

In [ ]:
sc.pl.umap(adata, color=["Ifng","cell_type"], frameon=False,gene_symbols="gene_name",legend_fontsize=7 , vmax="p99", cmap="Reds",add_outline=True)

In [ ]:
set(adata.obs.cell_type)

In [ ]:
order = [ 'COLON_Naive', 'COLON_Intermediate', 'COLON_Infg','COLON_Exhausted',


 'MPEC_Progenitor', 'MPEC_Intermediate',
 'MPEC_Effector',

 'SLEC_Progenitor', 'SLEC_Plastic', 'SLEC_Intermediate',
 'SLEC_Effector',
 'SLEC_Inf',



 'SLEC_Terminal']

In [ ]:
sc.pl.dotplot(adata, var_names=["Cxcr3"], groupby="cell_type", gene_symbols="gene_name", categories_order= order)

In [ ]:
sc.pl.dotplot(adata, var_names=["Ifng"], groupby="cell_type", gene_symbols="gene_name", categories_order= order)

In [ ]:
"Lcn2","Cxcl1","Cxcl5""Ccl20""Cxcl2","Cxcl9",

Cxcl9 --> Cxcr3 
Cxcl2 --> 
Ccl20 --> Ccr6 
CXCL5 --> Cxcr2 (not found)
Cxcl1 --> Cxcr2

In [ ]:
sc.pl.dotplot(adata, var_names=["Tnf","Cxcl10", "Cxcr3","Ccr6","Ccl5"], groupby="cell_type", gene_symbols="gene_name", categories_order= order)

In [ ]:
sc.pl.dotplot(adata, var_names=["Tnf","Cxcl10", "Cxcr3","Ccr6","Ccl5"], groupby="condition", gene_symbols="gene_name")

In [ ]:
sc.pl.dotplot(adata, var_names=["Tnf","Cxcl10", "Cxcr3","Ccr6","Ccl5"], groupby="origin", gene_symbols="gene_name")

In [ ]:
sc.pl.umap(adata, color="condition",groups="11mix", frameon=False)

In [ ]:
sc.pl.umap(adata, color="condition",groups="10mix", frameon=False)

In [ ]:
sc.pl.umap(adata, color="sample_id", groups= ["10mix_ICI1","10mix_ICI2"], frameon=False)

In [ ]:
# Query Omnipath and get PanglaoDB
markers = dc.get_resource('PanglaoDB',organism="mouse")
markers

In [ ]:
# Filter by canonical_marker and human
markers = markers[markers['mouse'] & markers['canonical_marker'] & (markers['mouse_sensitivity'] > 0.5)]

# Remove duplicated entries
markers = markers[~markers.duplicated(['cell_type', 'genesymbol'])]
markers

In [ ]:
adata.var = adata.var.set_index('gene_name')

In [ ]:
#adata.var_names_make_unique()

In [ ]:
adata.var

In [ ]:
dc.run_ora(
    mat=adata,
    net=markers,
    source='cell_type',
    target='genesymbol',
    min_n=1,
    verbose=True,
    use_raw=False
)

In [ ]:
acts = dc.get_acts(adata, obsm_key='ora_estimate')

# We need to remove inf and set them to the maximum value observed for pvals=0
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

acts